# Spark assignment 2: Collocations
As for the second part of the assignment, your task is to extract collocations: that is word combinations that occur together. For example, “high school” or “roman empire”.

To find collocations, you will use NPMI (normalized pointwise mutual information) metric.

PMI of two words, a & b, is defined as “PMI(a, b) = ln (P(ab) / (P(a) * P(b))”, where P(ab) is the probability of two words coming one after the other, and P(a) and P(b) are probabilities of words a & b respectively.

You will estimate probabilities with occurrence counts, that is “P(a) = # of occurrences of word a / total number of words”, and “P(ab) = # of occurrences of words ‘a b’ / total number of word pairs”.

To build an intuition behind the definition, see Reading material.

Therefore, rare combinations of coupled words have large PMI.

NPMI is computed as “NPMI(a, b) = PMI(a, b) / -ln P(ab)”. This normalizes the quantity to be within the range [-1; 1].

You task is a bit more complicated now:

Extract all the words, as in the previous task.
Filter out stopwords using the dictionary (/datasets/stop_words_en.txt ) (do not forget to convert words to the lowercase!)
Compute all bigrams (that is, pairs of consequent words)
Leave only bigrams with at least 500 occurrences
Compute NPMI for every bigram (note: when computing probabilities, you need unpruned counts!)
Sort word pairs by NPMI in the descending order
Print top 39 word pairs, with words delimited by the underscore “_”
For example,

roman_empire

south_africa

In [1]:
#map za ako imame [A, B, C...] A->A', B->B', C->C' (1->1)
#filter za ako imame [A, B, C...] A->, B->B, C->C (1->(0,1))
#flatMap za ako imame [A, B, C...] A->, B->(B',B''), C->C' (1->(0,*)) edna lista

from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))
import re


In [35]:
def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).lower().split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []
    
wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).map(parse_article) #one to one map

with open('/datasets/stop_words_en.txt', 'r') as f:
    stop_words = set(f.read().split())



In [36]:
#flatMap for filter to escape empty lists
filtered_words = wiki.map(lambda words : [word for word in words if word not in stop_words])


In [37]:

def bigrams(words):
    word_list=[]
    for inx, word in enumerate(words[:-1]):
         word_list.append((word+"_"+words[inx+1], 1))        
    return word_list

bigram_words = filtered_words.flatMap(lambda words : bigrams(words))


In [38]:
bigram_reduced = bigram_words.reduceByKey(lambda x, y : x+y).filter(lambda (key, value) : value>=500)

In [39]:
reduced_words = filtered_words.flatMap(lambda words : [(word,1) for word in words]).reduceByKey(lambda x,y : x+y).filter(lambda (key,value):value>=500)
reduced_word_collect = reduced_words.collect()
dict_words = {}
for key,value in reduced_word_collect:
    dict_words[key] = value

In [ ]:
total_words = len(dict_words)

In [ ]:
total_num_pairs = bigram_reduced.count()

In [ ]:
import math
def calculatePMI(key,value,dict_words,total_num_pairs, total_words):
    """PMI of two words, a & b, is defined as “PMI(a, b) = ln (P(ab) / (P(a) * P(b))”, 
    where P(ab) is the probability of two words coming one after the other, 
   and P(a) and P(b) are probabilities of words a & b respectively."""
   #You will estimate probabilities with occurrence counts, that is “P(a) = # of occurrences of word a / total number of words”, and “P(ab) = # of occurrences of words ‘a b’ / total number of word pairs”.
    word1, word2 = key.split("_")
    P_ab = value/float(total_num_pairs)
    P_a = dict_words[word1]/float(total_words)
    P_b = dict_words[word2]/float(total_words)
    ans = math.log(P_ab/(P_a*P_b))
    return (key, ans)

In [ ]:
ans = bigram_reduced.map(lambda (key,value) : calculatePMI(key, value,dict_words,total_num_pairs, total_words))

In [ ]:
answer = ans.map(lambda (key,value) : (value,key)).sortByKey(False).map(lambda (key,value) : (value, key)).take(39)

In [ ]:
for key, value in answer:
    print(key)